In [17]:
!pip install numpy==1.26.0

     |████████████████████████████████| 18.2 MB 4.1 MB/s            
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.0
    Uninstalling numpy-1.23.0:
      Successfully uninstalled numpy-1.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-gpu 2.6.0 requires absl-py~=0.10, but you have absl-py 2.1.0 which is incompatible.
tensorflow-gpu 2.6.0 requires flatbuffers~=1.12.0, but you have flatbuffers 24.3.25 which is incompatible.
tensorflow-gpu 2.6.0 requires h5py~=3.1.0, but you have h5py 3.12.1 which is incompatible.
tensorflow-gpu 2.6.0 requires keras~=2.6, but you have keras 3.7.0 which is incompatible.
tensorflow-gpu 2.6.0 requires numpy~=1.19.2, but you have numpy 1.26.0 which is incompatible.
tensorflow-gpu 2.6.0 requires six~=1.15.0, but you have six 1.16.0 which is incompatible.
tensorflow-gpu 2.6.0 requires typing-e

In [33]:
!pip install keras-tuner==1.0.3

     |████████████████████████████████| 96 kB 1.2 MB/s             
  Attempting uninstall: keras-tuner
    Found existing installation: keras-tuner 1.4.7
    Uninstalling keras-tuner-1.4.7:
      Successfully uninstalled keras-tuner-1.4.7


In [8]:
import os

import tensorflow as tf
import tensorflow.keras
import keras_tuner as kt

from sklearn.model_selection import train_test_split

In [14]:
(x_train, y_train), (x_test, y_test) = tensorflow.keras.datasets.mnist.load_data()

In [15]:
X_train = x_train.reshape(-1,28, 28, 1) 
X_test = x_test.reshape(-1,28,28,1)
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [16]:
# Normalize the pixel values to the range [0, 1]
X_train = X_train / 255.0
X_test = X_test / 255.0

In [17]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2)
print(X_train.shape)

(48000, 28, 28, 1)


In [6]:
class DeepTuner(kt.Tuner):
    def run_trial(self, trial, X, y, validation_data, **fit_kwargs):
        model = self.hypermodel.build(trial.hyperparameters)
        model.fit(X, y, batch_size=trial.hyperparameters.Choice(
            'batch_size', [16, 32, 64]), **fit_kwargs)


        X_val, y_val = validation_data
        eval_scores = model.evaluate(X_val, y_val)
        return {name: value for name, value in zip(
            model.metrics_names,
            eval_scores)}
    
    

In [108]:
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.Input(shape = X_train.shape[1:], name = 'inputs'))
    for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
              model.add(tf.keras.layers.Conv2D(hp.Int(
                  'units_{i}'.format(i=i), min_value=32, max_value=512, step=5), (3,3),activation='relu'))
    model.add(tf.keras.layers.Flatten())
    for i in range(hp.Int('n_connections', 1, 3)):
        model.add(tf.keras.layers.Dense(hp.Choice(f'n_nodes',
                                  values=[32,64,128,256,512]), activation = 'relu'))
    model.add(tf.keras.layers.Dense(10, activation='softmax', name = 'outputs'))
    model.compile(optimizer = 'adam',loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model

In [109]:
my_keras_tuner = DeepTuner(
    oracle=kt.oracles.BayesianOptimizationOracle(
        objective=kt.Objective('accuracy', 'max'),
        max_trials=10,
        seed=42),
    hypermodel=build_model,
    overwrite=True,
    project_name='my_keras_tuner')

### ++ 1st: n.0

In [10]:
model.fit(X_train, y_train, batch_size=32, epochs = 5)

Epoch 1/5
1500/1500 [==============================] - 16s 10ms/step - loss: 0.1993 - accuracy: 0.9404
Epoch 2/5
1500/1500 [==============================] - 15s 10ms/step - loss: 0.0828 - accuracy: 0.9744
Epoch 3/5
1500/1500 [==============================] - 15s 10ms/step - loss: 0.0668 - accuracy: 0.9792
Epoch 4/5
1500/1500 [==============================] - 15s 10ms/step - loss: 0.0560 - accuracy: 0.9831
Epoch 5/5
1500/1500 [==============================] - 15s 10ms/step - loss: 0.0520 - accuracy: 0.9840


### ++ 2nd: n.1

In [44]:
my_keras_tuner.search(
    X_train, y_train, validation_data=(X_val, y_val), epochs=3)

Trial 10 Complete [00h 00m 43s]
accuracy: 0.9783333539962769

Best accuracy So Far: 0.9880833625793457
Total elapsed time: 00h 07m 10s


In [45]:
best_hps = my_keras_tuner.get_best_hyperparameters(num_trials=10)[0]
model = build_model(best_hps)
model.summary()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_140 (Conv2D)          (None, 26, 26, 102)       1020      
_________________________________________________________________
conv2d_141 (Conv2D)          (None, 24, 24, 42)        38598     
_________________________________________________________________
conv2d_142 (Conv2D)          (None, 22, 22, 67)        25393     
_________________________________________________________________
conv2d_143 (Conv2D)          (None, 20, 20, 37)        22348     
_________________________________________________________________
conv2d_144 (Conv2D)          (None, 18, 18, 52)        17368     
_________________________________________________________________
flatten_23 (Flatten)         (None, 16848)             0         
_________________________________________________________________
dense_33 (Dense)             (None, 256)             

sequential_23/conv2d_140/Conv2D-0-Transpose

In [46]:
model.fit(X_train, y_train, batch_size=32, epochs = 5)

Epoch 1/5
1500/1500 [==============================] - 10s 7ms/step - loss: 0.1176 - accuracy: 0.9652
Epoch 2/5
1500/1500 [==============================] - 10s 7ms/step - loss: 0.0437 - accuracy: 0.9866
Epoch 3/5
1500/1500 [==============================] - 10s 7ms/step - loss: 0.0276 - accuracy: 0.9911
Epoch 4/5
1500/1500 [==============================] - 10s 7ms/step - loss: 0.0184 - accuracy: 0.9941
Epoch 5/5
1500/1500 [==============================] - 10s 7ms/step - loss: 0.0152 - accuracy: 0.9954


In [11]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 2s 5ms/step - loss: 0.0714 - accuracy: 0.9782


[0.07140526920557022, 0.9782000184059143]

In [47]:
saved_model_name = "./saved_model_1/"
tf.saved_model.save(model, saved_model_name)

INFO:tensorflow:Assets written to: ./saved_model_1/assets


### ++ 3rd: n.2

In [110]:
my_keras_tuner.search(
    X_train, y_train, validation_data=(X_val, y_val), epochs=3)

Trial 10 Complete [00h 02m 36s]
accuracy: 0.9780833125114441

Best accuracy So Far: 0.9879999756813049
Total elapsed time: 00h 26m 03s


In [111]:
best_hps = my_keras_tuner.get_best_hyperparameters(num_trials=10)[0]
model = build_model(best_hps)
model.summary()

Model: "sequential_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_210 (Conv2D)          (None, 26, 26, 382)       3820      
_________________________________________________________________
conv2d_211 (Conv2D)          (None, 24, 24, 102)       350778    
_________________________________________________________________
conv2d_212 (Conv2D)          (None, 22, 22, 222)       204018    
_________________________________________________________________
conv2d_213 (Conv2D)          (None, 20, 20, 67)        133933    
_________________________________________________________________
conv2d_214 (Conv2D)          (None, 18, 18, 127)       76708     
_________________________________________________________________
flatten_35 (Flatten)         (None, 41148)             0         
_________________________________________________________________
dense_50 (Dense)             (None, 256)             

In [112]:
model.fit(X_train, y_train, batch_size=32, epochs = 5)

Epoch 1/5
1500/1500 [==============================] - 27s 18ms/step - loss: 0.1241 - accuracy: 0.9637
Epoch 2/5
1500/1500 [==============================] - 27s 18ms/step - loss: 0.0458 - accuracy: 0.9855
Epoch 3/5
1500/1500 [==============================] - 28s 18ms/step - loss: 0.0289 - accuracy: 0.9908
Epoch 4/5
1500/1500 [==============================] - 27s 18ms/step - loss: 0.0194 - accuracy: 0.9943
Epoch 5/5
1500/1500 [==============================] - 27s 18ms/step - loss: 0.0134 - accuracy: 0.9959


In [113]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 2s 7ms/step - loss: 0.0497 - accuracy: 0.9872


[0.0497453473508358, 0.9872000217437744]

In [114]:
saved_model_name = "./saved_model_2/"
tf.saved_model.save(model, saved_model_name)

INFO:tensorflow:Assets written to: ./saved_model_2/assets


### ++ load model

In [4]:
!pip install tensorflow==2.5

     |████████████████████████████████| 454.4 MB 9.0 kB/s              
     |████████████████████████████████| 462 kB 98.5 MB/s            
     |████████████████████████████████| 4.0 MB 52.5 MB/s            
     |████████████████████████████████| 1.2 MB 31.0 MB/s            
     |████████████████████████████████| 5.5 MB 80.5 MB/s            
     |████████████████████████████████| 5.5 MB 82.6 MB/s            
     |████████████████████████████████| 5.5 MB 51.9 MB/s            
     |████████████████████████████████| 5.5 MB 67.5 MB/s            
     |████████████████████████████████| 5.5 MB 49.3 MB/s            
     |████████████████████████████████| 5.5 MB 66.9 MB/s            
     |████████████████████████████████| 5.5 MB 84.9 MB/s            
     |████████████████████████████████| 5.6 MB 46.3 MB/s            
     |████████████████████████████████| 5.5 MB 87.6 MB/s            
     |████████████████████████████████| 5.5 MB 48.0 MB/s            
     |█████████████████████████

In [2]:
!pip install protobuf==3.20

     |████████████████████████████████| 1.0 MB 3.9 MB/s            
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.1
    Uninstalling protobuf-5.29.1:
      Successfully uninstalled protobuf-5.29.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.5.0 requires absl-py<0.13,>=0.9, but you have absl-py 0.15.0 which is incompatible.


In [1]:
import tensorflow as tf
import numpy as np

In [2]:
saved_model = "./model/1/"
loaded_model = tf.saved_model.load(saved_model)

In [3]:
print(list(loaded_model.signatures.keys()))

['serving_default']


In [4]:
infer = loaded_model.signatures["serving_default"]
print(infer.structured_outputs)

{'outputs': TensorSpec(shape=(None, 10), dtype=tf.float32, name='outputs')}


In [18]:
input_test = tf.cast([X_test[0]], tf.float32)

In [19]:
pred = infer(inputs=input_test)

In [20]:
pred['outputs'].numpy()

array([[2.46230965e-12, 3.61849828e-10, 9.85459492e-10, 7.42842662e-11,
        2.25212574e-11, 5.32408929e-14, 1.28363862e-15, 1.00000000e+00,
        5.52970070e-09, 1.08810125e-08]], dtype=float32)

In [21]:
import numpy as np

index_of_max_value = np.argmax(pred['outputs'].numpy(), axis=1)

print(index_of_max_value)

[7]


In [22]:
np.argmax(y_test[0])

7

### ++ evaluation

In [89]:
input_test = tf.cast(X_test, tf.float32)
pred = infer(inputs=input_test)

In [94]:
pred['outputs'].numpy().shape

(10000, 10)

In [97]:
y_pred = pred['outputs'].numpy()

In [105]:
y_pred_classes = np.argmax(y_pred, axis=1)

In [106]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred_classes)
print("Accuracy:", accuracy)

Accuracy: 0.991


In [102]:
import tensorflow as tf
from sklearn.metrics import mean_squared_error

# Calculate mean squared error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.0014354988


In [107]:
from sklearn.metrics import classification_report

report = classification_report(np.argmax(y_test, axis=1), y_pred_classes)
print(report)

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       980
           1       1.00      1.00      1.00      1135
           2       0.99      0.99      0.99      1032
           3       0.99      1.00      0.99      1010
           4       0.99      0.99      0.99       982
           5       0.99      0.99      0.99       892
           6       0.98      0.99      0.99       958
           7       0.99      0.99      0.99      1028
           8       1.00      0.99      0.99       974
           9       0.99      0.99      0.99      1009

    accuracy                           0.99     10000
   macro avg       0.99      0.99      0.99     10000
weighted avg       0.99      0.99      0.99     10000



### ++ TFServing API

tensorflow_model_server --port=8501 --rest_api_port=8501 --model_name=saved_model_1 --model_base_path=/aiffel/aiffel/ex/13_mlops/model

In [23]:
!pip install tensorflow-serving-api==2.5

Reason for being yanked: incorrectly built.
  Attempting uninstall: tensorflow-serving-api
    Found existing installation: tensorflow-serving-api 2.6.0
    Uninstalling tensorflow-serving-api-2.6.0:
      Successfully uninstalled tensorflow-serving-api-2.6.0


In [24]:
import requests
import grpc
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

In [25]:
server_address = 'localhost:8501'

# Create a gRPC channel
channel = grpc.insecure_channel(server_address)

# Create a PredictionServiceStub
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

request = predict_pb2.PredictRequest()
request.model_spec.name = 'saved_model_1'
request.model_spec.signature_name = 'serving_default'

In [26]:
import numpy as np

image = X_test[0]
image = np.expand_dims(image, axis=0)
image = image.astype(np.float32)
inputs = tf.make_tensor_proto(image)

request.inputs['inputs'].CopyFrom(inputs)

In [27]:
response = stub.Predict(request=request)

In [28]:
response.outputs['outputs']

dtype: DT_FLOAT
tensor_shape {
  dim {
    size: 1
  }
  dim {
    size: 10
  }
}
float_val: 1.5016820252566898e-12
float_val: 5.004025194210726e-09
float_val: 1.2861796072627385e-09
float_val: 1.099483726285655e-09
float_val: 1.7177946176616388e-07
float_val: 1.6790818139090363e-13
float_val: 1.2225526959719796e-16
float_val: 0.9999985694885254
float_val: 4.687201737851865e-09
float_val: 1.3268081602291204e-06

In [29]:
tensor_np = tf.make_ndarray(response.outputs['outputs'])
max_index = np.argmax(tensor_np)

print(f"Predicted class: {max_index}")

Predicted class: 7


In [30]:
y_test[0].argmax()

7